# Setup

In [ ]:
%%capture
!rm -rf sample_data imagenet_class_index.json
!wget -O imagenet_class_index.json https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json

In [ ]:
import os
import json
import numpy as np

import tensorflow as tf
import tensorflow_datasets as tfds
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset#sample_from_datasets

import matplotlib as mpl
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

mpl.rcParams['figure.figsize'] = (8, 8)
mpl.rcParams['axes.grid'] = False

SEED = 301

In [ ]:
index_label_map = json.load(open('imagenet_class_index.json', 'r'))

# Load and Process Dataset

In [ ]:
# Helper function to preprocess the image so that it can be inputted in MobileNetV2
def preprocess(image, label=None):
  image = tf.cast(image, tf.float32)
  image = tf.image.resize(image, (224, 224))
  image = tf.keras.applications.mobilenet_v2.preprocess_input(image)

  return (image, label)

# Helper function to extract labels from probability vector
def get_imagenet_label(probs):
  return decode_predictions(probs, top=1)[0][0]

In [ ]:
# Construct a tf.data.Dataset
imagenet_r_train = tfds.load('imagenet_r', data_dir='data', split='test[:80%]', shuffle_files=True, as_supervised=True, try_gcs=True)
imagenet_r_test = tfds.load('imagenet_r', data_dir='data', split='test[80%:]', shuffle_files=True, as_supervised=True, try_gcs=True)

print("Training set size:", len(imagenet_r_train))
print("Test set size:", len(imagenet_r_test))

imagenet_r_train = imagenet_r_train.map(preprocess)
imagenet_r_test = imagenet_r_test.map(preprocess)

imagenet_r_train = imagenet_r_train.shuffle(len(imagenet_r_train))
imagenet_r_train = imagenet_r_train.cache()
imagenet_r_train = imagenet_r_train.batch(128)
imagenet_r_train = imagenet_r_train.prefetch(tf.data.AUTOTUNE)

imagenet_r_test = imagenet_r_test.shuffle(len(imagenet_r_test))
imagenet_r_test = imagenet_r_test.batch(128)
imagenet_r_test = imagenet_r_test.prefetch(tf.data.AUTOTUNE)

Training set size: 24000
Test set size: 6000


# Load Pre-trained Model

In [ ]:
learning_rate = 0.0001
epochs = 2

performance_history = {}

In [ ]:
pretrained_model = tf.keras.applications.MobileNetV2(include_top=True, weights='imagenet')
pretrained_model.trainable = False

pretrained_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    # metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
    metrics=['accuracy']
)

# ImageNet labels
decode_predictions = tf.keras.applications.mobilenet_v2.decode_predictions

In [ ]:
loss0, accuracy0 = pretrained_model.evaluate(imagenet_r_test)
print("BaseA Model Initial Loss: {:.4f}".format(loss0))
print("BaseA Model Initial Accuracy: {:.4f}".format(accuracy0))

47/47 [==============================] - 24s 176ms/step - loss: 5.2801 - accuracy: 0.2070
BaseA Model Initial Loss: 5.2801
BaseA Model Initial Accuracy: 0.2070


# Train Base Model

In [ ]:
base_model = tf.keras.applications.MobileNetV2(include_top=True)

base_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [ ]:
base_history = base_model.fit(imagenet_r_train, epochs=epochs, validation_data=imagenet_r_test)
performance_history['base'] = base_history.history

Epoch 1/2
188/188 [==============================] - 226s 931ms/step - loss: 2.4260 - accuracy: 0.5015 - val_loss: 2.1776 - val_accuracy: 0.5207
Epoch 2/2
188/188 [==============================] - 170s 905ms/step - loss: 1.1818 - accuracy: 0.7283 - val_loss: 2.0049 - val_accuracy: 0.5492


---

# Transfer Learning Model

In [ ]:
def get_a_n_b(pretrained_model, n_layers_to_copy, freeze, training_data, validation_data):
  local_model = None
  local_model = tf.keras.applications.MobileNetV2(include_top=True)
  local_model.trainable = True

  for i in range(n_layers_to_copy + 1):
    local_model.layers[i].set_weights(pretrained_model.layers[i].get_weights())
    if freeze:
      local_model.layers[i].trainable = False
    else:
      local_model.layers[i].trainable = True
    
  for i in range(n_layers_to_copy + 1, len(local_model.layers)):
    assert(local_model.layers[i].trainable == True)

  local_model.compile(
    optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate / 10),
    loss = tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
  )

  return local_model

In [ ]:
print('A_4blocks_B:')
A_4blocks_B = get_a_n_b(pretrained_model=pretrained_model, n_layers_to_copy=44, freeze=False, training_data=imagenet_r_train, validation_data=imagenet_r_test)
A_4blocks_B_history = A_4blocks_B.fit(imagenet_r_train, epochs=epochs, validation_data=imagenet_r_test)
performance_history['A_4blocks_B'] = A_4blocks_B_history.history

print('\nA_4blocks_B_freeze')
A_4blocks_B_freeze = get_a_n_b(pretrained_model=pretrained_model, n_layers_to_copy=44, freeze=True, training_data=imagenet_r_train, validation_data=imagenet_r_test)
A_4blocks_B_freeze_history = A_4blocks_B_freeze.fit(imagenet_r_train, epochs=epochs, validation_data=imagenet_r_test)
performance_history['A_4blocks_B_freeze'] = A_4blocks_B_freeze_history.history


print('\nA_8blocks_B')
A_8blocks_B = get_a_n_b(pretrained_model=pretrained_model, n_layers_to_copy=80, freeze=False, training_data=imagenet_r_train, validation_data=imagenet_r_test)
A_8blocks_B_history = A_8blocks_B.fit(imagenet_r_train, epochs=epochs, validation_data=imagenet_r_test)
performance_history['A_8blocks_B'] = A_8blocks_B_history.history

print('\nA_8blocks_B_freeze')
A_8blocks_B_freeze = get_a_n_b(pretrained_model=pretrained_model, n_layers_to_copy=80, freeze=True, training_data=imagenet_r_train, validation_data=imagenet_r_test)
A_8blocks_B_freeze_history = A_8blocks_B_freeze.fit(imagenet_r_train, epochs=epochs, validation_data=imagenet_r_test)
performance_history['A_8blocks_B_freeze'] = A_8blocks_B_freeze_history.history


print('\nA_12blocks_B')
A_12blocks_B = get_a_n_b(pretrained_model=pretrained_model, n_layers_to_copy=115, freeze=False, training_data=imagenet_r_train, validation_data=imagenet_r_test)
A_12blocks_B_history = A_12blocks_B.fit(imagenet_r_train, epochs=epochs, validation_data=imagenet_r_test)
performance_history['A_12blocks_B'] = A_12blocks_B_history.history

print('\nA_12blocks_B_freeze')
A_12blocks_B_freeze = get_a_n_b(pretrained_model=pretrained_model, n_layers_to_copy=115, freeze=True, training_data=imagenet_r_train, validation_data=imagenet_r_test)
A_12blocks_B_freeze_history = A_12blocks_B_freeze.fit(imagenet_r_train, epochs=epochs, validation_data=imagenet_r_test)
performance_history['A_12blocks_B_freeze'] = A_12blocks_B_freeze_history.history


print('\nA_16blocks_B')
A_16blocks_B = get_a_n_b(pretrained_model=pretrained_model, n_layers_to_copy=153, freeze=False, training_data=imagenet_r_train, validation_data=imagenet_r_test)
A_16blocks_B_history = A_16blocks_B.fit(imagenet_r_train, epochs=epochs, validation_data=imagenet_r_test)
performance_history['A_16blocks_B'] = A_16blocks_B_history.history

print('\nA_16blocks_B_freeze')
A_16blocks_B_freeze = get_a_n_b(pretrained_model=pretrained_model, n_layers_to_copy=153, freeze=True, training_data=imagenet_r_train, validation_data=imagenet_r_test)
A_16blocks_B_freeze_history = A_16blocks_B_freeze.fit(imagenet_r_train, epochs=epochs, validation_data=imagenet_r_test)
performance_history['A_16blocks_B_freeze'] = A_16blocks_B_freeze_history.history

A_4blocks_B:
Epoch 1/2
188/188 [==============================] - 178s 901ms/step - loss: 3.9894 - accuracy: 0.3102 - val_loss: 3.6189 - val_accuracy: 0.3557
Epoch 2/2
188/188 [==============================] - 168s 894ms/step - loss: 2.9072 - accuracy: 0.4353 - val_loss: 2.8863 - val_accuracy: 0.4323

A_4blocks_B_freeze
Epoch 1/2
188/188 [==============================] - 80s 390ms/step - loss: 3.9932 - accuracy: 0.3084 - val_loss: 3.6713 - val_accuracy: 0.3490
Epoch 2/2
188/188 [==============================] - 72s 385ms/step - loss: 2.9245 - accuracy: 0.4353 - val_loss: 2.9367 - val_accuracy: 0.4265

A_8blocks_B
Epoch 1/2
188/188 [==============================] - 178s 901ms/step - loss: 3.9894 - accuracy: 0.3102 - val_loss: 3.6192 - val_accuracy: 0.3557
Epoch 2/2
188/188 [==============================] - 168s 894ms/step - loss: 2.9072 - accuracy: 0.4353 - val_loss: 2.8865 - val_accuracy: 0.4328

A_8blocks_B_freeze
Epoch 1/2
188/188 [==============================] - 70s 343ms/ste

---

In [ ]:
print(performance_history)

{'base': {'loss': [2.426048994064331, 1.1817833185195923], 'accuracy': [0.5015000104904175, 0.7282916903495789], 'val_loss': [2.1775782108306885, 2.0048937797546387], 'val_accuracy': [0.5206666588783264, 0.5491666793823242]}, 'A_4blocks_B': {'loss': [3.989426851272583, 2.9071848392486572], 'accuracy': [0.3101666569709778, 0.43533334136009216], 'val_loss': [3.618946075439453, 2.8862955570220947], 'val_accuracy': [0.3556666672229767, 0.4323333203792572]}, 'A_4blocks_B_freeze': {'loss': [3.9931812286376953, 2.924473524093628], 'accuracy': [0.3084166646003723, 0.43525001406669617], 'val_loss': [3.6713178157806396, 2.93674898147583], 'val_accuracy': [0.3490000069141388, 0.42649999260902405]}, 'A_8blocks_B': {'loss': [3.98937726020813, 2.9072442054748535], 'accuracy': [0.3101666569709778, 0.43533334136009216], 'val_loss': [3.6191940307617188, 2.886542558670044], 'val_accuracy': [0.3556666672229767, 0.43283334374427795]}, 'A_8blocks_B_freeze': {'loss': [4.010810375213623, 2.9792683124542236],